In [13]:
from model_CRNN_seq import CRNN, SimpleCNN
import os
from PIL import Image
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.nn import CTCLoss
from torchvision import datasets, transforms
import torch.nn.functional as F
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.nn import CTCLoss
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from PIL import Image
import os

import os
from PIL import Image
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
abc = {
    "A": 0, "B": 1, "C": 2, "D": 3, "E": 4, "F": 5, "G": 6, "H": 7, "I": 8, "J": 9, "K": 10, "L": 11, "M": 12, "N": 13, "O": 14, "P": 15, "Q": 16, "R": 17, "S": 18, "T": 19, "U": 20, "V": 21, "W": 22, "X": 23, "Y": 24, "Z": 25, " ": 26
}

inverted_abc = {v: k for k, v in abc.items()}
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=ToTensor(), target_transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.target_transform = target_transform

        self.imgs = []
        for dirpath, dirnames, filenames in os.walk(self.root_dir):
            for dirname in dirnames:
                filenames = sorted(os.listdir(os.path.join(dirpath, dirname)))
                for filename in filenames:
                    path = os.path.join(dirpath, dirname, filename)
                    value_before_dot = filename.rsplit('.', 1)[0][-1]
                    target = self.get_target(dirname, value_before_dot)
                    length = len(filename)
                    self.imgs.append((path, target, length))


    def get_target(self, path, index):
        # Extract the specific character from the path and convert it to your target
        # This is just an example, adjust this to fit your needs
        target_char = path[int(index)]  
        if target_char in "- '":
            target= 26
        else:
            target = abc[target_char]  # Convert the character to an integer
        return target

    def __getitem__(self, index):
        path, target, length = self.imgs[index]
        img = Image.open(path).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return img, target, length

    def __len__(self):
        return len(self.imgs)

# Define the transformations
transform = transforms.Compose([
    transforms.Grayscale(),  # Convert to grayscale
    transforms.Resize((28, 28)),  # Resize to 32x32
    transforms.ToTensor(),  # Convert to tensor
])
# Create the datasets
val_dataset = CustomDataset(root_dir='/home/xnmaster/XNAPproject-grup_08/words_segmented_test', transform=transform)

# Create the data loaders
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

In [14]:
from model_CRNN_seq import CRNN, SimpleCNN
import os
from PIL import Image
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.nn import CTCLoss
from torchvision import datasets, transforms
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

abc = {
    "A": 0, "B": 1, "C": 2, "D": 3, "E": 4, "F": 5, "G": 6, "H": 7, "I": 8, "J": 9, "K": 10, "L": 11, "M": 12, "N": 13, "O": 14, "P": 15, "Q": 16, "R": 17, "S": 18, "T": 19, "U": 20, "V": 21, "W": 22, "X": 23, "Y": 24, "Z": 25, " ": 26
}

inverted_abc = {v: k for k, v in abc.items()}

# Create CNN model
cnn_model = SimpleCNN()
# Load the state dictionary
state_dict = torch.load('/home/xnmaster/XNAPproject-grup_08/model_CNN/results/dataset2/trained_model.pth', map_location=torch.device('cpu'))

# Create the CRNN model
crnn_model = CRNN(cnn_model, rnn_hidden_size=256, num_classes=27)  # 26 letters + 1 for blank

# Load the state dictionary
state_dict = torch.load('/home/xnmaster/XNAPproject-grup_08/model_CRNN/crnn_model.pth', map_location=torch.device('cpu'))



# Load the state dictionary
crnn_model.load_state_dict(state_dict)
# Set the model to evaluation mode
crnn_model.eval()


# Define the loss function and the optimizer
loss_function = CTCLoss(blank=26, zero_infinity=True)  # Assuming blank label is 26
optimizer = optim.Adam(crnn_model.parameters())


# testing loop

lib_images = []
lib_labels = []
lib_lengths = []

word_images = []
word_labels = []
word_lengths = []
lns = []
for i, (images, labels, lengths) in enumerate(val_loader):
    lns.append(lengths)
    # If the current word is finished, process the batch and start a new one
    if i > 0 and lns[i-1] != lns[i]:
        word_images = torch.stack(word_images)
        word_labels = torch.stack(word_labels)
        word_lengths = torch.stack(word_lengths)
        # Move the images and labels to the GPU if available
        lib_images.append(word_images)
        lib_labels.append(word_labels) 
        lib_lengths.append(word_lengths)
        
        word_images = []
        word_labels = []
        word_lengths = []
    word_images.append(images)
    word_labels.append(labels)
    word_lengths.append(lengths)
test_loss = 0
correct = 0
    # process the batch
lib_images = lib_images[1:]
lib_labels = lib_labels[1:]
lib_lengths = lib_lengths[1:]

In [17]:
with torch.no_grad():
    for word_images, word_labels, word_lengths in zip(lib_images, lib_labels, lib_lengths):
        word_images = word_images.to(device)
        word_labels = word_labels.to(device)
        word_lengths = word_lengths.to(device)
        outputs = crnn_model(word_images)
        outputs = outputs.transpose(0, 1)
        input_lengths = torch.tensor([outputs.size(0)], dtype=torch.long)        
        target_lengths = torch.tensor([word_labels.size(0)], dtype=torch.long)
        word_labels = word_labels.unsqueeze(0)
        word_labels = word_labels.view(-1)
        loss = loss_function(outputs, word_labels, input_lengths, target_lengths)
        test_loss += loss.item()
        pred = outputs.argmax(dim=2)
        pred_str = ''
        for l in pred:
            pred_str += inverted_abc[l.item()]
        label_str = ''
        for l in word_labels:
            label_str += inverted_abc[l.item()]
        if pred_str == label_str:
            correct += 1
test_loss /= len(val_loader)
accuracy = correct / len(val_loader)
print(f'Loss: {test_loss}, Accuracy: {accuracy}')




Loss: -4.3991090329362885e-05, Accuracy: 0.00013979822456254806


{0: 'A',
 1: 'B',
 2: 'C',
 3: 'D',
 4: 'E',
 5: 'F',
 6: 'G',
 7: 'H',
 8: 'I',
 9: 'J',
 10: 'K',
 11: 'L',
 12: 'M',
 13: 'N',
 14: 'O',
 15: 'P',
 16: 'Q',
 17: 'R',
 18: 'S',
 19: 'T',
 20: 'U',
 21: 'V',
 22: 'W',
 23: 'X',
 24: 'Y',
 25: 'Z',
 26: ' '}